In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### open and read the shakespeare file in lower case

In [2]:
txt = []
directory = "/kaggle/input/shakespeare-txt"
sentence = []
for file in os.listdir(directory):
    with open(os.path.join(directory , file) , "r") as f:
        txt = f.read().lower()
        

In [3]:
len(txt)

5436475

#### take only 60000 caracters to not face ram problems

In [4]:
data = txt[0:60000]

#### import the tokenizer

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

2025-09-12 14:32:47.995663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757687568.171856      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757687568.217079      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### fit the tokenizer on the data in a word level tokenization 

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
words_vocab = len(tokenizer.word_index) + 1

#### we have 2361 unique words in data

In [7]:
print(words_vocab)

2361


#### apply the tokenizer on the data so we end up with an array of indexes 

In [8]:
sequence = tokenizer.texts_to_sequences([data])[0]

#### so we have 10373 word and 10373 indexes

In [9]:
len(sequence)

10373

#### the second most frequent word in the dataset we tokenized is and

In [10]:
word = tokenizer.index_word[1]
print(word)

and


#### so we split data in a way so that the model trains on 7 words and predict the 8th and a step by one

In [11]:
x = []
y = []
seq_length = 7
for i in range(seq_length, len(sequence)):
    if i < seq_length + 5:  # only show the first 5 examples
        print(sequence[i-seq_length:i], "->", sequence[i])
    x.append(sequence[i-seq_length:i])
    y.append(sequence[i])

[35, 901, 902, 228, 195, 402, 8] -> 560
[901, 902, 228, 195, 402, 8, 560] -> 125
[902, 228, 195, 402, 8, 560, 125] -> 403
[228, 195, 402, 8, 560, 125, 403] -> 126
[195, 402, 8, 560, 125, 403, 126] -> 270


#### convert the x (7 words) and y (the 8th word) to numpy array to match the inout expected by the model

In [12]:
x = np.array(x)
y = np.array(y)

#### print the shape of X

In [13]:
x.shape

(10366, 7)

In [14]:
len(x)

10366

#### use train test splpit mehtos for better evaluation

In [15]:
from sklearn.model_selection import train_test_split
x_tain, x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2)

#### import the layers to build the model architecture

In [16]:
from tensorflow.keras.models import Model , Sequential 
from tensorflow.keras.layers import Dense , Embedding , LSTM , add , Dropout
from tensorflow.keras.utils import plot_model

so each word in the sequence is embedded for semantic meaning then fed into the LSTM the sequence word by word till the last word where we have an output vector of the sentence then a dropout by 0.3 to reduce overfitting and feed the result to a dense layer then do a probability distribtion of all the words the one with the highest probability is chosen

compile , train and save the model

In [17]:
model = Sequential()
model.add(Embedding(words_vocab, 256, input_length=seq_length))               # optional
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.3))                 # recommended
model.add(Dense(words_vocab, activation="softmax"))


model.compile(
   loss="sparse_categorical_crossentropy",  # handles integer labels
   optimizer="adam",
   metrics=["accuracy"]
)

model.fit(x, y, epochs=50, batch_size=128, verbose=1)

model.save("lstm_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1757687580.777491      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1757687580.778303      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/50


I0000 00:00:1757687585.516376     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


81/81 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.0214 - loss: 7.3314
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0254 - loss: 6.3828
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0248 - loss: 6.2318
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0271 - loss: 6.1483
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0312 - loss: 6.0912
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0390 - loss: 6.0255
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0432 - loss: 5.9215
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0478 - loss: 5.8134
Epoch 9/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0543 - loss: 5.7124
Epoch 10/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0598 - loss: 5.5458
Epoch 11/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0684 - loss: 5.3683
Epoch 12/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0834 - loss: 5.1762


we got a perfect accuracy 0.9855 and a perfect loss also 0.18

evluating the model on the test data and it's perfect there's no overfitting accuracy 0.99 and loss 0.06 which is perfect

In [18]:
model.evaluate(x_test , y_test)

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9991 - loss: 0.0691


[0.07384383678436279, 0.9985535144805908]

a function to generate words from a sentence that we feed to the model

In [ ]:
def generate_poem(seq_length , input_text):
    result = input_text
    for _ in range(seq_length):
        word_index = tokenizer.texts_to_sequences([input_text])[0]
        token_list = word_index[-seq_length:]
        token_list = np.array(token_list).reshape(1 ,-1)
        predicted_probs = model.predict(token_list , verbose=0)
        predicted_index = np.argmax(predicted_probs , axis=-1)[0]
        predicted_word = tokenizer.index_word.get(predicted_index, "")
        input_text += " " + predicted_word
        result += " " + predicted_word

    return result

a test generate 100 words from this sentence 

In [20]:
print(generate_poem(100 ,"From fairest creatures we desire increase"))

From fairest creatures we desire increase things besmeared or merit alone i time i cold alone look look despite the sight in my sight thou what thou mayst thou me look look look look look look look look look look look look look look look look look look look look look look 'tis such my heart the spirit that thou such my heart mine own heart mine own desert and by thy pleasure mine eye the spirit in mine own love's silent such mine eye and thou such a living day a living hue a tattered sight of me the heart of thy love and i


another test with 10 words

In [21]:
print(generate_poem(10 , "Those hours that with gentle work did frame"))

Those hours that with gentle work did frame no frame no fair of many is it is so


In [22]:
print(generate_poem(10 , "Which happies those that pay the willing"))

Which happies those that pay the willing loan loan loan loan that's for thy self to honour


In [24]:
print(generate_poem(10, """From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:"""))


From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel: thy sweet use thy name and found thy ripe thoughts
